# Analyse des retards au départ de Paris-CDG – Été 2022  
**Projet Data Analyst – Simulation et étude opérationnelle**

Ce notebook présente un projet complet de data analyse basé sur une situation réelle :  
la crise opérationnelle de l’été 2022 à l’aéroport Paris-Charles de Gaulle (CDG), marquée par une reprise rapide du trafic et de fortes perturbations (retards, pertes de bagages, sous-effectifs).

### 🎯 Objectifs du projet
- **Simuler un dataset réaliste** de 500 vols au départ de CDG entre juin et août 2022.  
- **Analyser la ponctualité (OTP)** selon différents axes :  
  - plage horaire,  
  - terminal,  
  - type de vol,  
  - cause de retard,  
  - avant / après un incident bagages simulé début juillet 2022.  
- **Identifier les sources principales de perturbation** et les zones critiques dans les opérations sol.  
- **Proposer des actions concrètes** pour améliorer la fiabilité du hub et anticiper les pics de trafic.

### 🧰 Compétences mises en œuvre
- Simulation et préparation de données (Python, Pandas).  
- Nettoyage, transformation et enrichissement de données.  
- Analyse descriptive (OTP, distributions, segmentation).  
- Visualisation (Matplotlib).  
- Interprétation orientée métier (opérations aériennes).  

Ce projet constitue une base solide pour un portfolio professionnel dans le domaine **Data / Aviation / Opérations**.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Options d'affichage pour mieux lire les DataFrame
pd.set_option("display.max_columns", 50)

# Définition des chemins du projet
PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

print("Racine du projet :", PROJECT_ROOT)
print("Dossier data     :", DATA_DIR)



Racine du projet : /Users/godwincherubin/Documents/GitHub/cdg-summer-2022-delay-analysis
Dossier data     : /Users/godwincherubin/Documents/GitHub/cdg-summer-2022-delay-analysis/data


## 1. Construction pédagogique du jeu de données

Dans un contexte réel, les données proviendraient du data warehouse d’une compagnie aérienne  
(ex : systèmes opérationnels, données AODB, logs bagages, fichiers équipages, etc.).

Dans ce projet, je reconstruis un jeu de données **pédagogique mais réaliste**, inspiré de la crise opérationnelle de l’été 2022 à Paris-CDG :

- **vols_cdg_ete2022** : liste de vols au départ de CDG entre juin et août 2022 (500 vols simulés),
- **causes_retard** : cause principale de retard pour chaque vol (bagages, handling, ATC, météo…).

Ces données permettent de reproduire un raisonnement professionnel d’analyste opérations :
- impact d’un incident bagages,
- évolution de la ponctualité (OTP),
- analyse par terminal, par plage horaire, par type de vol,
- identification des zones critiques dans le hub.

La première étape consiste à générer les **dates des vols**, base du dataset.


In [2]:
# Nombre total de vols à simuler
n_flights = 500

# Générateur aléatoire (pour avoir toujours les mêmes résultats)
rng = np.random.default_rng(42)

# Période de l'étude : 1er juin → 31 août 2022
date_range = pd.date_range("2022-06-01", "2022-08-31", freq="D")

# On tire aléatoirement 500 dates dans cette période
dates_vol = rng.choice(date_range, size=n_flights)

# Création d'un DataFrame de base
df = pd.DataFrame({
    "date_vol": pd.to_datetime(dates_vol).strftime("%Y-%m-%d")
})

df.head()


,date_vol
0,2022-06-09
1,2022-08-11
2,2022-07-31
3,2022-07-11
4,2022-07-10


### 1.1 Attribution des caractéristiques opérationnelles des vols

À partir des dates générées, j’ajoute maintenant les informations opérationnelles propres à chaque vol :

- terminal de départ (2E, 2F, 2G),
- zone/porte d’embarquement,
- type de vol (court-courrier / long-courrier),
- destination et région,
- type d’appareil.

Ces éléments reflètent la structure d’un vrai hub comme CDG et permettront d’analyser la ponctualité selon différents segments.


In [3]:
# Attribution des terminaux (répartition réaliste pour CDG)
terminals = rng.choice(["2E", "2F", "2G"], size=n_flights, p=[0.5, 0.35, 0.15])

# Définition des zones d’embarquement selon le terminal
zone_map = {
    "2E": ["2E_L", "2E_M"],
    "2F": ["2F1", "2F2"],
    "2G": [None],  # navette bus, pas de zone classique
}
zone_gate = [rng.choice(zone_map[t]) for t in terminals]

# Type de vol : 70% court-courrier, 30% long-courrier
type_vol = rng.choice(["CC", "LC"], size=n_flights, p=[0.7, 0.3])

# Destinations et appareils cohérents par segment
cc_destinations = [
    ("LHR", "Europe", "A320"),
    ("AMS", "Europe", "A320"),
    ("FCO", "Europe", "A321"),
    ("BCN", "Europe", "A320"),
    ("MAD", "Europe", "A321"),
    ("MUC", "Europe", "E190"),
    ("LYS", "Europe", "E190"),
]

lc_destinations = [
    ("JFK", "Amériques", "B777"),
    ("LAX", "Amériques", "B777"),
    ("JNB", "Afrique", "B777"),
    ("DXB", "Asie", "A350"),
    ("SIN", "Asie", "A350"),
]

# Affectation destination/région/appareil vol par vol
destinations = []
regions = []
appareils = []

for tv in type_vol:
    if tv == "CC":
        d, r, a = cc_destinations[rng.integers(0, len(cc_destinations))]
    else:
        d, r, a = lc_destinations[rng.integers(0, len(lc_destinations))]
    destinations.append(d)
    regions.append(r)
    appareils.append(a)

# Ajout au DataFrame
df["terminal"] = terminals
df["zone_gate"] = zone_gate
df["type_vol"] = type_vol
df["destination"] = destinations
df["region_destination"] = regions
df["type_appareil"] = appareils

df.head()


,date_vol,terminal,zone_gate,type_vol,destination,region_destination,type_appareil
0,2022-06-09,2F,2F2,CC,MAD,Europe,A321
1,2022-08-11,2G,None,LC,LAX,Amériques,B777
2,2022-07-31,2E,2E_L,CC,AMS,Europe,A320
3,2022-07-11,2E,2E_L,LC,DXB,Asie,A350
4,2022-07-10,2E,2E_L,CC,AMS,Europe,A320


### 1.2 Simulation des heures de départ et des retards (incident du 2 juillet)

Je simule maintenant :

- l’heure de départ prévue (entre 06:00 et 22:00),
- le retard au départ (en minutes),
- l’heure réelle de départ.

La distribution des retards est volontairement différente **avant et après le 2 juillet 2022**,  
pour représenter un incident majeur sur le système de tri bagages à CDG.


In [4]:
# Date de l'incident bagages
incident_date = pd.to_datetime("2022-07-02")

# Conversion de la date du vol en datetime
df["date_vol_dt"] = pd.to_datetime(df["date_vol"])

# Indicateur : vol avant ou après l'incident
avant_incident = df["date_vol_dt"] < incident_date

# Heure de départ prévue : entre 06:00 et 22:00
heures = rng.integers(6, 22, size=n_flights)
minutes = rng.choice(range(0, 60, 5), size=n_flights)
df["heure_depart_prevue"] = [f"{h:02d}:{m:02d}" for h, m in zip(heures, minutes)]

# Grille de retards possibles (en minutes)
delay_bins = [0, 5, 10, 15, 20, 30, 45, 60, 90, 120]

# Distribution des retards avant l'incident (plutôt correcte)
prob_before = np.array([0.35, 0.25, 0.15, 0.10, 0.07, 0.04, 0.02, 0.01, 0.005, 0.005])
prob_before /= prob_before.sum()

# Distribution des retards après l'incident (plus dégradée)
prob_after = np.array([0.15, 0.15, 0.15, 0.10, 0.10, 0.10, 0.08, 0.07, 0.05, 0.05])
prob_after /= prob_after.sum()

# Attribution du retard en fonction de la date (avant/après incident)
df["retard_depart_min"] = np.where(
    avant_incident,
    rng.choice(delay_bins, size=n_flights, p=prob_before),
    rng.choice(delay_bins, size=n_flights, p=prob_after)
)

# Calcul de l'heure réelle de départ
dt_prev = pd.to_datetime(df["date_vol"] + " " + df["heure_depart_prevue"])
dt_real = dt_prev + pd.to_timedelta(df["retard_depart_min"], unit="m")
df["heure_depart_reelle"] = dt_real.dt.strftime("%H:%M")

df.head()


,date_vol,terminal,zone_gate,type_vol,destination,region_destination,type_appareil,date_vol_dt,heure_depart_prevue,retard_depart_min,heure_depart_reelle
0,2022-06-09,2F,2F2,CC,MAD,Europe,A321,2022-06-09,13:30,5,13:35
1,2022-08-11,2G,None,LC,LAX,Amériques,B777,2022-08-11,20:05,60,21:05
2,2022-07-31,2E,2E_L,CC,AMS,Europe,A320,2022-07-31,18:35,5,18:40
3,2022-07-11,2E,2E_L,LC,DXB,Asie,A350,2022-07-11,10:40,15,10:55
4,2022-07-10,2E,2E_L,CC,AMS,Europe,A320,2022-07-10,09:40,120,11:40


### 1.3 Identifiant de vol, classe de retard et informations calendrier

Chaque vol reçoit maintenant :

- un **identifiant unique** (`flight_id`),
- une **classe de retard** (0–15 min, 15–30 min, 30–60 min, 60+ min),
- des informations calendrier :
  - **jour de la semaine**,
  - **mois**.

Ces variables serviront à segmenter les analyses (par jour, par période, par typologie de retard).


In [5]:
# Fonction de classification du retard
def classer_retard(m):
    if m <= 15:
        return "A_0_15"
    if m <= 30:
        return "B_15_30"
    if m <= 60:
        return "C_30_60"
    return "D_60_plus"

# Classe de retard
df["classe_retard"] = df["retard_depart_min"].apply(classer_retard)

# Identifiant unique du vol
df["flight_id"] = [f"ES{str(i+1).zfill(3)}" for i in range(n_flights)]

# Jour de la semaine
df["jour_semaine"] = df["date_vol_dt"].dt.day_name()

# Mois (numéro 6–7–8)
df["mois"] = df["date_vol_dt"].dt.month

df.head()


,date_vol,terminal,zone_gate,type_vol,destination,region_destination,type_appareil,date_vol_dt,heure_depart_prevue,retard_depart_min,heure_depart_reelle,classe_retard,flight_id,jour_semaine,mois
0,2022-06-09,2F,2F2,CC,MAD,Europe,A321,2022-06-09,13:30,5,13:35,A_0_15,ES001,Thursday,6
1,2022-08-11,2G,None,LC,LAX,Amériques,B777,2022-08-11,20:05,60,21:05,C_30_60,ES002,Thursday,8
2,2022-07-31,2E,2E_L,CC,AMS,Europe,A320,2022-07-31,18:35,5,18:40,A_0_15,ES003,Sunday,7
3,2022-07-11,2E,2E_L,LC,DXB,Asie,A350,2022-07-11,10:40,15,10:55,A_0_15,ES004,Monday,7
4,2022-07-10,2E,2E_L,CC,AMS,Europe,A320,2022-07-10,09:40,120,11:40,D_60_plus,ES005,Sunday,7


### 1.4 Attribution des causes principales de retard (interne vs externe)

Pour chaque vol, j’attribue une **cause principale de retard** parmi une liste inspirée des opérations réelles à CDG :

- BAGAGES  
- HANDLING  
- SECURITE  
- ROTATION  
- CREW  
- ATC (Air Traffic Control)  
- METEO  
- AUTRE  

Les causes sont également regroupées en deux catégories :

- **Interne** (bagages, handling, sûreté, rotation, équipage)  
- **Externe** (ATC, météo, autres événements)

Cette variable est essentielle pour déterminer quelles actions relèvent :
- de l’aéroport / compagnie (actions internes),  
- des contraintes extérieures (ATC, météo…).


In [6]:
# Liste des causes possibles
causes = ["BAGAGES", "HANDLING", "SECURITE", "ROTATION", "CREW",
          "ATC", "METEO", "AUTRE"]

# Probabilités inspirées de l'été 2022 (internal issues dominantes)
cause_prob = [0.22, 0.20, 0.15, 0.10, 0.08, 0.10, 0.08, 0.07]
cause_prob = np.array(cause_prob) / np.sum(cause_prob)

# Attribution aléatoire pondérée
df_causes = pd.DataFrame({
    "flight_id": df["flight_id"],
    "code_retard_principal": rng.choice(causes, size=n_flights, p=cause_prob)
})

# Détection interne/externe
internes = {"BAGAGES", "HANDLING", "SECURITE", "ROTATION", "CREW"}
df_causes["cause_interne"] = df_causes["code_retard_principal"].isin(internes)

df_causes.head()


,flight_id,code_retard_principal,cause_interne
0,ES001,ROTATION,True
1,ES002,BAGAGES,True
2,ES003,CREW,True
3,ES004,HANDLING,True
4,ES005,BAGAGES,True


### 1.5 Export des tables finales au format CSV

La préparation du jeu de données est terminée.  
Je construis maintenant deux tables prêtes pour l’analyse :

- `vols_cdg_ete2022.csv` :  
  une ligne = un vol au départ de CDG, avec ses caractéristiques opérationnelles et son retard au départ.

- `causes_retard.csv` :  
  une ligne = un vol, avec sa cause principale de retard et un indicateur **interne / externe**.

Ces fichiers seront utilisés dans un second notebook dédié à l’analyse des retards et à la construction des KPI.


In [7]:
# On s'assure que l'indicateur cause_interne est bien en entier (0/1)
df_causes["cause_interne"] = df_causes["cause_interne"].astype(int)

# Table vols : sélection des colonnes utiles pour l'analyse
colonnes_vols = [
    "flight_id",
    "date_vol",
    "jour_semaine",
    "mois",
    "heure_depart_prevue",
    "heure_depart_reelle",
    "retard_depart_min",
    "classe_retard",
    "terminal",
    "zone_gate",
    "destination",
    "region_destination",
    "type_vol",
    "type_appareil",
]

df_vols = df[colonnes_vols].copy()

# Export CSV
df_vols.to_csv(DATA_DIR / "vols_cdg_ete2022.csv", index=False)
df_causes.to_csv(DATA_DIR / "causes_retard.csv", index=False)

# Vérification des fichiers écrits
df_vols_check = pd.read_csv(DATA_DIR / "vols_cdg_ete2022.csv")
df_causes_check = pd.read_csv(DATA_DIR / "causes_retard.csv")

print("DATA_DIR utilisé :", DATA_DIR)
print("vols_cdg_ete2022 :", df_vols_check.shape)
print("causes_retard    :", df_causes_check.shape)


DATA_DIR utilisé : /Users/godwincherubin/Documents/GitHub/cdg-summer-2022-delay-analysis/data
vols_cdg_ete2022 : (500, 14)
causes_retard    : (500, 3)


## Conclusion — Construction du dataset

Le jeu de données simulant les opérations de l’été 2022 à CDG est maintenant entièrement construit.

Ce notebook a permis de :
- générer **500 vols** répartis sur la période **juin–août 2022**,  
- simuler les caractéristiques opérationnelles clés (terminal, destination, type de vol, appareil…),
- intégrer un **incident majeur le 2 juillet** impactant fortement les retards,
- attribuer une **cause principale de retard** (interne ou externe) pour chaque vol,
- préparer deux tables exploitables pour l’analyse :

1. `vols_cdg_ete2022.csv`  
2. `causes_retard.csv`

Ces fichiers forment la base du travail analytique qui sera réalisé dans le **Notebook 02 "02_analysis_vols_cdg_ete2022.ipynb"**, où je mesurerai :

- la ponctualité départ (OTP),
- les zones critiques (terminaux, plages horaires),
- l’impact de l’incident du 2 juillet,
- la répartition réelle des causes de retard,
- et les axes d’amélioration opérationnelle pour éviter un scénario similaire.

Le Notebook 02 intitulé "02_analysis_vols_cdg_ete2022.ipynb" constituera l’analyse principale du projet, avec visualisations, KPI, et interprétation business.
